In [1]:
import gensim
from numpy import linalg
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm_notebook as tqdm
import time
from random import shuffle
import sys
import nltk 
from nltk.corpus import wordnet 
import gc
from collections import defaultdict
import random
import json
import os

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.test.utils import datapath, get_tmpfile

from gensim.test.utils import datapath
%load_ext autotime

# load embeddings and normalizing them

In [2]:
model_glove = glove2word2vec('glove.42B.300d.txt','gensim_glove_300d.txt') ## needed for glove embeddings only

time: 8.57 s


In [3]:
model_glove = gensim.models.KeyedVectors.load_word2vec_format("gensim_glove_300d.txt", binary=False) ## needed for only glove

time: 4min 40s


In [4]:
def generate_norm_embedding(model, output_path):
    temp_file = open(output_path,'wb')
    temp_file.write(str.encode(str(len(model.vocab))+' '+str(model.vector_size)+'\n'))
    
    for each_word in tqdm(model.vocab):
        temp_file.write(str.encode(each_word+' '))
        temp_file.write(model[each_word]/linalg.norm(model[each_word]))
        temp_file.write(str.encode('\n'))
    
    temp_file.close()

time: 5.63 ms


In [5]:
generate_norm_embedding(model_glove,'glove_norm_300.mod')


time: 24.6 s


### Load the required word-embeddings

In [6]:
model_gn = gensim.models.KeyedVectors.load_word2vec_format('glove_norm_300.mod',binary=True) # ss

time: 28.5 s


In [7]:
current_model = model_gn 

time: 1.71 ms


# Loading the antonyms

In [8]:
list_antonym = []

with open('Antonym_sets/LenciBenotto.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))


with open('Antonym_sets/LenciBenotto.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('Antonym_sets/EVALution.val') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
with open('Antonym_sets/EVALution.test') as fp:
    for line in fp:
        parts = line.split()
        if parts[3]=='antonym':
            word1 = parts[0].split('-')[0]
            word2 = parts[1].split('-')[0]
            if word1 in current_model and word2 in current_model:
                list_antonym.append((word1.strip().lower(), word2.strip().lower()))
                
                
list_antonym = list(dict.fromkeys(list_antonym).keys())

time: 20.1 ms


In [9]:
similarity_matrix = defaultdict(list)
for each_pair in tqdm(list_antonym):
    word1 = each_pair[0]
    word2 = each_pair[1]
    if word1 < word2:
        similarity_matrix[word1].append(word2)
    else:
        similarity_matrix[word2].append(word1)
    
all_similarity = defaultdict(dict)
for each_key in tqdm(similarity_matrix):
    for each_value in similarity_matrix[each_key]:
#         cosine_similarity([current_model[each_key]]
        all_similarity[each_key][each_value] = abs(cosine_similarity([current_model[each_key]],[current_model[each_value]])[0][0])
    
final_antonym_list = []
for index_counter, each_key in enumerate(tqdm(all_similarity)):
#     print(each_key,all_similarity[each_key])
    listofTuples = sorted(all_similarity[each_key].items() ,  key=lambda x: x[1])
#     print(listofTuples)
    final_antonym_list.append((each_key, listofTuples[0][0]))
print(len(final_antonym_list))

list_antonym = final_antonym_list


1468
time: 457 ms


## Decide on the size of the antonym vector

In [10]:
num_antonym = 1468

time: 1.69 ms


In [11]:
## Find the anatonym difference vectors
antonymy_vector = []
for each_word_pair in list_antonym:
    antonymy_vector.append(current_model[each_word_pair[0]]- current_model[each_word_pair[1]])
antonymy_vector = np.array(antonymy_vector)
print(antonymy_vector.shape)

(1468, 300)
time: 10 ms


## Subset Dimension Selection Method

In [12]:
import random

from scipy.spatial.distance import cosine as scipy_cosine
random.seed(42)

t1 = np.array(antonymy_vector)
dimension_similarity_matrix = defaultdict(dict)
for index_1, each_dim1 in enumerate(tqdm(t1)):
    for index_2, each_dim2 in enumerate(t1):
        dimension_similarity_matrix[index_1][index_2] = abs(1-scipy_cosine(each_dim1, each_dim2))
        
        
def get_set_score(final_list, each_dim):
    final_output = 0.0
    for each_vec in final_list:
        final_output += dimension_similarity_matrix[each_vec][each_dim]
    return final_output/(len(final_list))
        
def select_subset_dimension(dim_vector, num_dim):
    working_list = np.array(dim_vector)
    
    working_position_index = [i for i in range(working_list.shape[0])]
    final_position_index = []
    

    print('working list is ready, shape', working_list.shape)
    sel_dim = random.randrange(0, working_list.shape[0])

    final_position_index.append(sel_dim)
    
    working_position_index.remove(sel_dim)

    
    for test_count in tqdm(range(num_dim-1)):
        min_dim = None
        min_score = 1000
        for temp_index, each_dim in enumerate(working_position_index):
#             print(each_dim)
            temp_score = get_set_score(final_position_index, each_dim)
            if temp_score< min_score:
                min_score= temp_score
                min_dim = each_dim
        print(test_count,min_dim)
        final_position_index.append(min_dim)
        working_position_index.remove(min_dim)
#         print(working_list.shape, len(final_list))
    return final_position_index


time: 1min 7s


## Generate the ORTHOGONAL DIMENSION Order

In [13]:
num_antonym = 1443
orthogonal_antonymy_vector =np.array(select_subset_dimension(antonymy_vector, num_antonym))  
print(orthogonal_antonymy_vector.shape)

working list is ready, shape (1468, 300)


0 243
1 493
2 626
3 670
4 87
5 1219
6 1156
7 485
8 1445
9 1171
10 1433
11 258
12 1380
13 1389
14 1462
15 1205
16 1040
17 166
18 745
19 393
20 261
21 473
22 1122
23 1378
24 1349
25 794
26 1174
27 1300
28 167
29 389
30 1347
31 1262
32 460
33 1426
34 544
35 1152
36 1226
37 1402
38 325
39 1026
40 663
41 961
42 484
43 717
44 232
45 5
46 750
47 1108
48 488
49 290
50 1325
51 1341
52 664
53 1311
54 1382
55 1357
56 1418
57 343
58 1014
59 1387
60 348
61 1003
62 1427
63 430
64 759
65 28
66 1385
67 291
68 404
69 535
70 751
71 318
72 420
73 202
74 1305
75 695
76 944
77 129
78 920
79 11
80 67
81 1251
82 726
83 450
84 161
85 1074
86 747
87 463
88 1290
89 1004
90 1410
91 1237
92 1352
93 349
94 1083
95 895
96 1274
97 146
98 592
99 221
100 1016
101 1103
102 784
103 1400
104 529
105 1072
106 889
107 545
108 805
109 962
110 1033
111 713
112 1020
113 418
114 771
115 1176
116 391
117 843
118 901
119 299
120 60
121 311
122 1110
123 1
124 973
125 489
126 157
127 1210
128 644
129 818
130 1009
131 555
132 1054


1003 182
1004 126
1005 214
1006 514
1007 1135
1008 504
1009 366
1010 249
1011 452
1012 1005
1013 646
1014 2
1015 1320
1016 897
1017 1017
1018 461
1019 570
1020 958
1021 513
1022 1123
1023 1302
1024 1071
1025 954
1026 1061
1027 642
1028 293
1029 1224
1030 310
1031 1119
1032 128
1033 952
1034 227
1035 1109
1036 1340
1037 22
1038 1066
1039 467
1040 1345
1041 1024
1042 225
1043 252
1044 580
1045 326
1046 828
1047 636
1048 800
1049 399
1050 1222
1051 300
1052 1381
1053 127
1054 112
1055 482
1056 1051
1057 471
1058 1239
1059 847
1060 883
1061 1390
1062 1157
1063 1178
1064 431
1065 319
1066 1118
1067 1214
1068 183
1069 1045
1070 307
1071 503
1072 426
1073 809
1074 1013
1075 1244
1076 618
1077 383
1078 733
1079 170
1080 414
1081 51
1082 632
1083 753
1084 681
1085 1388
1086 43
1087 77
1088 55
1089 852
1090 1076
1091 500
1092 470
1093 1337
1094 965
1095 1021
1096 701
1097 15
1098 373
1099 1030
1100 941
1101 566
1102 1360
1103 152
1104 213
1105 454
1106 595
1107 456
1108 916
1109 387
1110 597
111

## Generate the RANDOM DIMENSION Order

In [14]:
random_antonym_vector = [i for i in range(len(antonymy_vector))]
random.shuffle(random_antonym_vector)
print(len(random_antonym_vector))

1468
time: 3.58 ms


## Generate the MAXIMUM VARIANCE DIMENSION Order

In [15]:
embedding_size = antonymy_vector.shape[0]   
print('The embedding size is', embedding_size)


variance_antonymy_vector_inverse = np.linalg.pinv(np.transpose(antonymy_vector))

embedding_matrix = []


total_words = 0
for each_word in tqdm(current_model.vocab):
    total_words += 1

    new_vector = np.matmul(variance_antonymy_vector_inverse,current_model[each_word])
    
    embedding_matrix.append(new_vector)


The embedding size is 1468



time: 30.7 s


In [16]:
del new_vector

time: 1.71 ms


In [17]:
variance_list = []

embedding_matrix = np.array(embedding_matrix)

for each_dimension in tqdm(range(embedding_matrix.shape[1])):
    variance_list.append(np.var(embedding_matrix[:,each_dimension]))


time: 47.5 s


In [18]:
variance_antonymy_vector = [each for each in sorted(range(len(variance_list)), key=lambda i: variance_list[i], reverse=True)]

time: 2.63 ms


In [19]:
del embedding_matrix

time: 211 ms


In [20]:
del variance_list

time: 1.67 ms


# Transformtaion to polar space

In [22]:
def transform_to_antonym_space(current_model, output_file_path, binary, current_antonymy_vector_inverse):
    embedding_size = current_antonymy_vector_inverse.shape[0]   ##CHANGE THIS ACCORDINGLY!!!
    print('New model size is',len(current_model.vocab), embedding_size)

    temp_file = None
    
    if binary:
        temp_file = open(output_file_path,'wb')
        temp_file.write(str.encode(str(len(current_model.vocab))+' '+str(embedding_size)+'\n'))
    else:
        temp_file = open(output_file_path,'w')
        temp_file.write(str(len(current_model.vocab))+' '+str(embedding_size)+'\n')

    total_words = 0
    for each_word in tqdm(current_model.vocab):
        total_words += 1
        if binary:
            temp_file.write(str.encode(each_word+' '))
        else:
            temp_file.write(each_word+' ')

        new_vector = np.matmul(current_antonymy_vector_inverse,current_model[each_word])

        new_vector = new_vector/linalg.norm(new_vector)

        
        
        if binary:
            temp_file.write(new_vector)
            temp_file.write(str.encode('\n'))
        else:
            temp_file.write(str(new_vector))
            temp_file.write('\n')


    temp_file.close()



time: 4.45 ms


# Standard normal transform 

In [23]:
def standard_normal_dist_model(model, new_filename):
    embedding_matrix = []
    embedding_vocab = []

    temp_file = open(new_filename,'wb')
    temp_file.write(str.encode(str(model.vectors.shape[0])+' '+str(model.vectors.shape[1])+'\n'))
    
    for each_word in tqdm(model.vocab):
        embedding_matrix.append(model[each_word])
        embedding_vocab.append(each_word)
    
    embedding_matrix = np.array(embedding_matrix)
    
    print('The shape of embedding matrix is {}'.format(embedding_matrix.shape))
    
    norm_embedding_matrix = (embedding_matrix - embedding_matrix.mean(0))/ embedding_matrix.std(0)
    
    for word_counter, each_word in enumerate(tqdm(embedding_vocab)):
#         assert each_word==embedding_vocab[word_counter],'Not matching!!!'
        
        temp_file.write(str.encode(each_word+' '))
        new_vector = norm_embedding_matrix[word_counter]
        temp_file.write(new_vector)
        temp_file.write(str.encode('\n'))
        
    del embedding_matrix
    del embedding_vocab
    temp_file.close()

time: 3.93 ms


## Compute the task score for different dimension size

In [24]:
def generate_embedding_path(current_model, embedding_path, binary, antonym_vector, curr_dim):
    curr_antonym_vector = antonymy_vector[antonym_vector[:curr_dim]]
    curr_antonymy_vector_inverse = np.linalg.pinv(np.transpose(curr_antonym_vector))
    transform_to_antonym_space(current_model, embedding_path, binary,curr_antonymy_vector_inverse)

time: 3.05 ms


In [25]:
import subprocess
dim_size = 500 # Number of POLAR dimenions
antonym_vector_method = random_antonym_vector # orthogonal_antonymy_vector, variance_antonymy_vector

time: 2.14 ms


In [27]:
curr_dim = dim_size

embedding_path = 'Downstream Task/embeddings/rand_antonym_'+str(curr_dim)+'.bin'
generate_embedding_path(current_model, embedding_path,True,random_antonym_vector, curr_dim)

print('loading the model')
temp_model = gensim.models.KeyedVectors.load_word2vec_format(embedding_path, binary=True)
print('loading done..')

std_nrml_embedding_path = 'Downstream Task/embeddings/rand_antonym_gl_'+str(curr_dim)+'_StdNrml.bin'
standard_normal_dist_model(temp_model, std_nrml_embedding_path)

######### Task specific code....
command_list = ['python3', 
                    'Downstream Task/TREC/classify_task.py',
                     std_nrml_embedding_path,
                     '2',
                     'Downstream Task/TREC/data/qa_train_X.pickle',
                     'Downstream Task/TREC/data/qa_train_y.pickle',
                     'Downstream Task/TREC/data/qa_val_X.pickle',
                     'Downstream Task/TREC/data/qa_val_y.pickle',
                     'Downstream Task/TREC/data/qa_test_X.pickle',
                     'Downstream Task/TREC/data/qa_test_y.pickle'
                    ]
result = subprocess.run(command_list, stdout=subprocess.PIPE)
print(result.stdout)
print('finished sentiment task')
################
del temp_model
    
os.remove(std_nrml_embedding_path)
os.remove(embedding_path)

New model size is 1917494 500



loading the model
loading done..



The shape of embedding matrix is (1917494, 500)



b'0.966\n'
finished sentiment task
time: 4min 39s


# Code snippet to use for other downstream tasks

In [ ]:

## word analogy....
######################################################################   
    
    del temp_model
    
    temp_model_nrml = gensim.models.KeyedVectors.load_word2vec_format(std_nrml_embedding_path, binary=True)
    accuracy = temp_model_nrml.evaluate_word_analogies(datapath('questions-words.txt'))[0]
    
    print(accuracy)
    final_output_wa.write(str(curr_dim)+'\tstdnrml\t')
    final_output_wa.write(str(accuracy))
    final_output_wa.write('\n')
    final_output_wa.flush()
    
    del temp_model_nrml
######################################################################
### Sentiment classification    
    command_list = ['python3', 
                    'Downstream Task/sentiment/classify_sentiment.py',
                     std_nrml_embedding_path,
                     '2',
                     'Downstream Task/sentiment/data/sentiment_train_X.p',
                     'Downstream Task/sentiment/data/sentiment_train_y.p',
                     'Downstream Task/sentiment/data/sentiment_val_X.p',
                     'Downstream Task/sentiment/data/sentiment_val_y.p',
                     'Downstream Task/sentiment/data/sentiment_test_X.p',
                     'Downstream Task/sentiment/data/sentiment_test_y.p'
                    ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished sentiment task')




######################################################################
### Question classification
    command_list = ['python3', 
                    'Downstream Task/TREC/classify_task.py',
                     std_nrml_embedding_path,
                     '2',
                     'Downstream Task/TREC/data/qa_train_X.pickle',
                     'Downstream Task/TREC/data/qa_train_y.pickle',
                     'Downstream Task/TREC/data/qa_val_X.pickle',
                     'Downstream Task/TREC/data/qa_val_y.pickle',
                     'Downstream Task/TREC/data/qa_test_X.pickle',
                     'Downstream Task/TREC/data/qa_test_y.pickle'
                    ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished TREC task')
    

#####################################################################
### Discriminative attribute identification
    command_list = ['python3', 
                    'Downstream Task/Discrim_Attr/classify_discrim_attr_TASK.py',
                    std_nrml_embedding_path,
                    '2',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_train_X.p',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_train_y.p',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_val_X.p',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_val_y.p',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_test_X.p',
                    'Downstream Task/Discrim_Attr/data/discrim_attr_test_y.p'
                   ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished Discr task')
    
######################################################################
### Computer news classification
    command_list = ['python3',
                    'Downstream Task/newsgroups/classify.py',
                    std_nrml_embedding_path,
                    '2',
                    'Downstream Task/newsgroups/data/news_computer_train_X.p',
                    'Downstream Task/newsgroups/data/news_computer_train_y.p',
                    'Downstream Task/newsgroups/data/news_computer_val_X.p',
                    'Downstream Task/newsgroups/data/news_computer_val_y.p',
                    'Downstream Task/newsgroups/data/news_computer_test_X.p',
                    'Downstream Task/newsgroups/data/news_computer_test_y.p'
    ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished compter nc task')

######################################################################   
### Religious news classification
    command_list = ['python3',
                    'Downstream Task/newsgroups/classify.py',
                    std_nrml_embedding_path,
                    '2',
                    'Downstream Task/newsgroups/data/news_religion_train_X.p',
                    'Downstream Task/newsgroups/data/news_religion_train_y.p',
                    'Downstream Task/newsgroups/data/news_religion_val_X.p',
                    'Downstream Task/newsgroups/data/news_religion_val_y.p',
                    'Downstream Task/newsgroups/data/news_religion_test_X.p',
                    'Downstream Task/newsgroups/data/news_religion_test_y.p'
    ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished Religion nc task')

######################################################################
### sports news classification
    command_list = ['python3',
                    'Downstream Task/newsgroups/classify.py',
                    std_nrml_embedding_path,
                    '2',
                    'Downstream Task/newsgroups/data/news_sports_train_X.p',
                    'Downstream Task/newsgroups/data/news_sports_train_y.p',
                    'Downstream Task/newsgroups/data/news_sports_val_X.p',
                    'Downstream Task/newsgroups/data/news_sports_val_y.p',
                    'Downstream Task/newsgroups/data/news_sports_test_X.p',
                    'Downstream Task/newsgroups/data/news_sports_test_y.p'
    ]
    result = subprocess.run(command_list, stdout=subprocess.PIPE)
    print(result.stdout)
    print('finished Sports nc task')

    
######################################################################  
### noun phrase bracketing
    res = []
    for _ in range(10):
        command_list = ['python3',
                        'Downstream Task/np_bracketing/classify_bracketing.py',
                        std_nrml_embedding_path,
                        '2',
                        'Downstream Task/np_bracketing/data/npbracketing_train_X'+str(_)+'.pickle',
                        'Downstream Task/np_bracketing/data/npbracketing_train_y'+str(_)+'.pickle',
                        'Downstream Task/np_bracketing/data/npbracketing_val_X'+str(_)+'.pickle',
                        'Downstream Task/np_bracketing/data/npbracketing_val_y'+str(_)+'.pickle',
                        'Downstream Task/np_bracketing/data/npbracketing_test_X'+str(_)+'.pickle',
                        'Downstream Task/np_bracketing/data/npbracketing_test_y'+str(_)+'.pickle'
        ]

        result = subprocess.run(command_list, stdout=subprocess.PIPE)
        print(result.stdout)
        res.append(float(result.stdout.decode('utf-8').strip()))
    
    #print(res)
    m_s = np.mean(res)
    print(m_s)
